In [5]:
import sys
import os
from keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from skimage.io import imread
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd

In [6]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
K.clear_session()

In [8]:
data_entrenamiento = './train'
data_validacion = './test'

In [9]:
#Parametros red neuronal

epocas = 20
altura, longitud = 100, 100
batch_size = 32
pasos = 200
pasos_validacion = 50
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 29
lr = 0.0005

In [10]:
images_df = pd.read_csv(r'.\\file_target.csv')

images_df = images_df.rename(columns={'archivo': 'filenames',
                       'target': 'category'})

images_df = images_df.sort_values('category')

images_df['category'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'del', 'nothing', 'space'], dtype=object)

In [11]:
entrenamiento, validacion = train_test_split(images_df, test_size = 0.19)

entrenamiento = entrenamiento.reset_index(drop=True)

entrenamiento = entrenamiento.sort_values('category')

validacion = validacion.reset_index(drop=True)

validacion = validacion.sort_values('category')

In [12]:
#Pre-procesamiento de las imágenes
#Primero creamos un generador que dice como vamos a preprocesar la info

entrenamiento_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

validacion_datagen = ImageDataGenerator(rescale = 1.0/255.0)

imagen_entrenamiento = entrenamiento_datagen.flow_from_dataframe(entrenamiento,
                                                                 data_entrenamiento,
                                                                 target_size=(altura, longitud),
                                                                 x_col='filenames',
                                                                y_col='category',
                                                                batch_size=batch_size,
                                                                class_mode='categorical')

imagen_validacion = validacion_datagen.flow_from_dataframe(validacion,
                                                           data_entrenamiento,
                                                          target_size=(altura, longitud),
                                                           x_col='filenames',
                                                            y_col='category',
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

Found 7290 validated image filenames belonging to 29 classes.
Found 1710 validated image filenames belonging to 29 classes.


In [13]:
cnn=Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())

cnn.add(Dense(256, activation='relu'))

cnn.add(Dropout(0.5))

cnn.add(Dense(clases, activation='softmax'))

In [14]:
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

In [14]:
cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = './modelo/'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Epoch 1/20
200/200 [==============================] - 27s 133ms/step - loss: 1.6955 - accuracy: 0.4206 - val_loss: 1.0881 - val_accuracy: 0.6556
Epoch 2/20
200/200 [==============================] - 28s 141ms/step - loss: 1.6705 - accuracy: 0.4299 - val_loss: 1.0871 - val_accuracy: 0.6712
Epoch 3/20
200/200 [==============================] - 31s 154ms/step - loss: 1.6640 - accuracy: 0.4285 - val_loss: 1.0807 - val_accuracy: 0.6500
Epoch 4/20
200/200 [==============================] - 30s 150ms/step - loss: 1.6618 - accuracy: 0.4285 - val_loss: 1.0418 - val_accuracy: 0.6837
Epoch 5/20
200/200 [==============================] - 30s 148ms/step - loss: 1.6466 - accuracy: 0.4392 - val_loss: 1.1705 - val_accuracy: 0.6075
Epoch 6/20
200/200 [==============================] - 29s 146ms/step - loss: 1.6333 - accuracy: 0.4409 - val_loss: 0.9776 - val_accuracy: 0.7138
Epoch 7/20
200/200 [==============================] - 29s 144ms/step - loss: 1.6277 - accuracy: 0.4464 - val_loss: 1.0722 - val_ac

In [15]:
modelo='./modelo/modelo.h5'

cnn = load_model(modelo)

cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = './modelo/'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Epoch 1/20
200/200 [==============================] - 29s 145ms/step - loss: 1.2796 - accuracy: 0.5472 - val_loss: 0.6987 - val_accuracy: 0.7794
Epoch 2/20
200/200 [==============================] - 28s 141ms/step - loss: 1.2522 - accuracy: 0.5638 - val_loss: 0.5599 - val_accuracy: 0.8344
Epoch 3/20
200/200 [==============================] - 29s 143ms/step - loss: 1.2235 - accuracy: 0.5648 - val_loss: 0.5868 - val_accuracy: 0.8331
Epoch 4/20
200/200 [==============================] - 29s 143ms/step - loss: 1.2573 - accuracy: 0.5588 - val_loss: 0.6087 - val_accuracy: 0.7994
Epoch 5/20
200/200 [==============================] - 29s 143ms/step - loss: 1.2120 - accuracy: 0.5757 - val_loss: 0.5598 - val_accuracy: 0.8319
Epoch 6/20
200/200 [==============================] - 28s 142ms/step - loss: 1.2189 - accuracy: 0.5693 - val_loss: 0.5900 - val_accuracy: 0.8263
Epoch 7/20
200/200 [==============================] - 29s 144ms/step - loss: 1.1941 - accuracy: 0.5820 - val_loss: 0.5322 - val_ac